In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 3 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

07 November 2022 ; 10:02:39


# Features, model

In [3]:
features = ['VORP','WS','OWS','BPM','PER','PTS','FT','FTA','OBPM','FG','USG%','Seed','MP']#,'GS'] # GS !!!
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/mvp/mvp_data.csv')

# Load to-be-predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') 

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,Precious Achiuwa,C,23,TOR,10,0,21.8,3.0,7.8,0.385,...,7.3,116.84,109.98,6.86,9.0,117.62,108.99,8.63,0,1
1,OG Anunoby,SF,25,TOR,10,10,34.9,6.1,12.7,0.480,...,7.3,116.84,109.98,6.86,9.0,117.62,108.99,8.63,0,0
2,Dalano Banton,PG,23,TOR,9,0,7.9,1.4,2.9,0.500,...,7.3,116.84,109.98,6.86,9.0,117.62,108.99,8.63,0,1
3,Scottie Barnes,PF,21,TOR,9,9,31.4,6.1,12.6,0.487,...,7.3,116.84,109.98,6.86,9.0,117.62,108.99,8.63,0,0
4,Khem Birch,C,30,TOR,4,0,8.5,0.8,1.3,0.600,...,7.3,116.84,109.98,6.86,9.0,117.62,108.99,8.63,0,1


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] >= season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player' ,'Tm', 'PER', 'VORP', 'WS', 'PTS', 'Seed', 'Share']]
res.head()

,Player,Tm,PER,VORP,WS,PTS,Seed,Share
0,Giannis Antetokounmpo,MIL,33.4,10.250,18.450000,32.6,1,0.897366
1,Luka Dončić,DAL,36.7,11.275,22.550000,36.0,6,0.748194
2,Donovan Mitchell,CLE,24.9,8.200,16.400000,31.4,2,0.505355
3,Damian Lillard,POR,27.1,8.200,13.120000,31.0,4,0.463265
4,Stephen Curry,GSW,27.9,8.200,13.666667,31.0,13,0.436557


In [7]:
res.to_csv(path + f'Results/mvp/results_week_{week}.csv', index = None)